In [197]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [198]:
nltk.download('brown')
nltk.download('universal_tagset')
nltk.download('punkt')
nltk.download('universal_tagset')
from nltk.corpus import brown


[nltk_data] Downloading package brown to
[nltk_data]     /Users/manojbhadu/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/manojbhadu/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/manojbhadu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/manojbhadu/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [199]:
brown_corpus = list(nltk.corpus.brown.tagged_sents(tagset='universal'))

In [25]:
print(brown_corpus[1])

[('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('of', 'ADP'), ('Atlanta', 'NOUN'), ("''", '.'), ('for', 'ADP'), ('the', 'DET'), ('manner', 'NOUN'), ('in', 'ADP'), ('which', 'DET'), ('the', 'DET'), ('election', 'NOUN'), ('was', 'VERB'), ('conducted', 'VERB'), ('.', '.')]


In [26]:
len(brown_corpus)  #total no. of sentences in dataset

57340

In [201]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [37]:
train_data,test_data =train_test_split(brown_corpus,test_size=0.20,random_state = 0)

In [40]:
print(len(train_data), len(test_data))

45872 11468


In [53]:
#print word with its tag for 1st sentence
for sent in brown_corpus[:1]:
    for tupl in sent:
        print(tupl)

('The', 'DET')
('Fulton', 'NOUN')
('County', 'NOUN')
('Grand', 'ADJ')
('Jury', 'NOUN')
('said', 'VERB')
('Friday', 'NOUN')
('an', 'DET')
('investigation', 'NOUN')
('of', 'ADP')
("Atlanta's", 'NOUN')
('recent', 'ADJ')
('primary', 'NOUN')
('election', 'NOUN')
('produced', 'VERB')
('``', '.')
('no', 'DET')
('evidence', 'NOUN')
("''", '.')
('that', 'ADP')
('any', 'DET')
('irregularities', 'NOUN')
('took', 'VERB')
('place', 'NOUN')
('.', '.')


In [54]:
#tranforming dataset into each word and his tag
train_tagged_words = [ tupl for sent in train_data for tupl in sent ]
test_tagged_words = [ tupl for sent in test_data for tupl in sent ]

In [59]:
print(train_tagged_words[:3])
print(test_tagged_words[:3])
print(len(train_tagged_words))
print(len(test_tagged_words))

[('This', 'DET'), ('will', 'VERB'), ('mean', 'VERB')]
[('Muscle', 'NOUN'), ('weakness', 'NOUN'), ('did', 'VERB')]
933117
228075


In [71]:
#printing tags and words(vocab size) in our data

tags = {tag for words,tag in train_tagged_words}
print(len(tags))

vocab = {word for word,tag in train_tagged_words}
print(len(vocab))

print(tags)


12
50749
{'ADV', 'VERB', 'DET', 'PRT', 'PRON', 'ADP', 'ADJ', 'NOUN', 'X', 'CONJ', 'NUM', '.'}


In [ ]:
unique_tags = list(all_tags)
tpm_dict = {}
for tag in unique_tags:
    for suc_tag in unique_tags:
        tpm_dict[tag][suc_tag] = 0
for sent in training_sents:
    for pos in range(len(sent)-1):
        tpm_dict[sent[pos][1]][sent[pos+1][1]]+=1


### Emission and Transition Probabilities

In [144]:
def emission_prob(word,tag,datset,alpha=0.2):               #add alpha smoothing
    tag_count=0             #no. of times given tag appear in datset
    for i in range(len(datset)):
        if datset[i][1] == tag:
            tag_count+=1
    word_with_tag=0          #no. of times given word tagged as given tag
    for i in range(len(datset)):
        if datset[i][1] == tag and datset[i][0] == word :
            word_with_tag+=1
    
    vocab = {word for word,tag in train_tagged_words}
    V = len(vocab)
    prob = (word_with_tag+alpha)/(tag_count+(alpha*V))
    return prob

In [145]:
p = emission_prob('will','VERB',train_tagged_words)
p

0.01105080230279275

In [132]:
def transition_prob(tag1, tag2, dataset):             #calculating tag2 given tag1
    all_tags = [word_tag[1] for word_tag in dataset]
    count_tag1 = len([tag for tag in all_tags if tag==tag1])
    tag2_with_tag1 = 0
    for i in range(len(all_tags)-1):
        if tag1 == all_tags[i] and tag2 == all_tags[i+1]:
            tag2_with_tag1+=1
    
    prob = (tag2_with_tag1)/(count_tag1)
    return prob
    

In [133]:
t1 = train_tagged_words[1][1]
print(t1)
t2 = train_tagged_words[3][1]
print(t2)
tp = transition_prob(t1,t2,train_tagged_words)
print(t2,"given",t1,'is',tp)

VERB
ADP
ADP given VERB is 0.16916213616736814


In [134]:
#Table for transtion probability
#(i,j) element of table represent the transtion from i to j
all_tags = {word_tag[1] for word_tag in train_tagged_words}
all_tags = list(all_tags)
n=len(all_tags)
transition_matrix = np.zeros((n,n), dtype = float)
for i in range(n):
    for j in range(n):
        transition_matrix[i,j] = transition_prob(all_tags[i],all_tags[j],train_tagged_words)

In [135]:
pd_transition = pd.DataFrame(transition_matrix, columns = all_tags, index=all_tags)

In [136]:
pd_transition

,ADV,VERB,DET,PRT,PRON,ADP,ADJ,NOUN,X,CONJ,NUM,.
ADV,0.095735,0.239483,0.073005,0.028674,0.048522,0.143969,0.136873,0.032400,0.000089,0.017497,0.013328,0.170426
VERB,0.103937,0.184265,0.162771,0.065287,0.055028,0.169162,0.057545,0.097818,0.000157,0.014318,0.008936,0.080778
DET,0.017361,0.064863,0.005990,0.001945,0.009926,0.009008,0.239990,0.626158,0.001436,0.000645,0.009753,0.012925
PRT,0.035475,0.622861,0.082523,0.011699,0.007170,0.090280,0.019540,0.035894,0.000084,0.012202,0.004780,0.077491
PRON,0.054189,0.705815,0.017431,0.023014,0.008109,0.056109,0.009726,0.009095,0.000025,0.011646,0.001086,0.103754
ADP,0.015533,0.041217,0.454722,0.014117,0.070403,0.020222,0.082304,0.258943,0.000498,0.001915,0.030200,0.009926
ADJ,0.009475,0.017168,0.005926,0.019619,0.003728,0.087847,0.057149,0.655236,0.000520,0.037010,0.007084,0.099238
NOUN,0.026672,0.158508,0.016115,0.017945,0.020212,0.244749,0.013095,0.149988,0.000329,0.060259,0.007848,0.284281
X,0.008515,0.051088,0.005676,0.008515,0.008515,0.052034,0.003784,0.056764,0.494797,0.026490,0.000000,0.283822
CONJ,0.090341,0.193969,0.151436,0.024816,0.066632,0.073732,0.112160,0.246629,0.000586,0.000228,0.018596,0.020875


In [ ]:
#table for emission probability
#(i,j) element of table is emission probability of word vocab[j] in state i
# vocab = list({word_tag[0] for word_tag in train_tagged_words})
# all_tags = list({word_tag[1] for word_tag in train_tagged_words})
# V = len(vocab)
# n = len(all_tags)
# emission_matrix = np.zeros((n,V))
# for i in range(n):
#     for j in range(V):
#         emission_matrix[i,j] = emission_prob(vocab[j],all_tags[i],train_tagged_words)

### Viterbi Algoritham

In [161]:
def viterbi(sentence, training_data):
    states = []
    all_tags = list({word_tag[1] for word_tag in training_data})
    T = len(sentence)
    for i in range(T):
        p=[]
        for tag in all_tags:
            #compute transition probabilities
            if i == 0:
                trans_prob = pd_transition.loc['.',tag]
            else:
                trans_prob = pd_transition.loc[states[-1],tag]
            #compute emission probabilities
            emis_prob = emission_prob(sentence[i],tag,train_tagged_words)
            #state probability P(Sk->Sk+1)*P(Ok|Sk) , trans_prob*emis_prob
            state_probability = trans_prob*emis_prob
            p.append(state_probability)
        
        max_state_prob = max(p)                               #max probability
        argmax_state_prob = all_tags[p.index(max_state_prob)]      #state with max probabiltiy

        states.append(argmax_state_prob)
    return list(zip(sentence,states))

In [162]:
random.seed(101)

In [163]:
test = [i[0] for i in test_data[1]]

In [164]:
ans = viterbi(test, train_tagged_words)

In [165]:
ans

[('He', 'PRON'),
 ('replaced', 'VERB'),
 ('the', 'DET'),
 ('flashlight', 'NOUN'),
 ('where', 'ADV'),
 ('it', 'PRON'),
 ('had', 'VERB'),
 ('been', 'VERB'),
 ('stowed', 'PRT'),
 (',', '.'),
 ('got', 'VERB'),
 ('into', 'ADP'),
 ('his', 'DET'),
 ('own', 'ADJ'),
 ('car', 'NOUN'),
 ('and', 'CONJ'),
 ('backed', 'VERB'),
 ('it', 'PRON'),
 ('out', 'PRT'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('garage', 'NOUN'),
 ('.', '.')]

In [1]:
#For Accuracy calculation , but takes long time

#rndom = [random.randint(1,len(test_data)) for x in range(10)]
#test_run = [test_data[i] for i in rndom]
# test_run = test_data[5]
# test_run_base = [tup for sent in test_run for tup in sent]
# test_tagged_words = [tup[0] for sent in test_run for tup in sent]

# start = time.time()
# tagged_seq = viterbi(test_tagged_words,train_tagged_words)
# end = time.time()
# difference = end-start

# print("Time taken in seconds: ", difference)

# # accuracy
# check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

# accuracy = len(check)/len(tagged_seq)
# print('Viterbi Algorithm Accuracy: ',accuracy*100)

In [193]:
test = [i[0] for i in test_data[5]]
pred_tags=viterbi(test,train_tagged_words)

In [210]:
pred_tag = [i[1] for i in pred_tags]
real_tag = [i[1] for i in test_data[5]]
confusion_matrix(real_tag,pred_tag)


array([[3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 6, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 4]])

In [209]:
real_tag

['NOUN',
 'NOUN',
 'NOUN',
 '.',
 'PRON',
 'VERB',
 'DET',
 'NOUN',
 'NOUN',
 '.',
 'VERB',
 'ADV',
 'PRT',
 'VERB',
 'DET',
 'ADJ',
 'NOUN',
 'ADP',
 'DET',
 'NOUN',
 'ADV',
 'PRON',
 'VERB',
 'ADP',
 'NUM',
 '.']